In [1]:
from PySide6.QtGui import *
from PySide6.QtWidgets import *
from PySide6.QtCore import *
from PySide6.QtSerialPort import QSerialPort, QSerialPortInfo
import sys
import cv2
import os
import numpy as np
from importlib import reload
from functools import partial
from detect import detectDefects, drawDefects

import ui.ui_mainwindow
reload(ui.ui_mainwindow)
from ui.ui_mainwindow import *

from serial import *
import serial.tools.list_ports

ports = sorted(serial.tools.list_ports.comports())
for port in ports:
    try:
        qport = QSerialPort(port.device)
        qport.close()
    except Exception as e:
        print(f"Could not close: {port.device}: {e}")

class MainWindow(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.initialized = False
        self.setupUi(self)
        self.setWindowTitle("Texas Investigator")
        self.setWindowIcon(QIcon('icon.jpg'))
        QApplication.setWindowIcon(QIcon('icon.jpg'))
        
        self.grabberAngleBox.editingFinished.connect(self.setGrabberAngle)
        self.speedBox.editingFinished.connect(self.setSpeed)
        self.identifyButton.clicked.connect(self.identifyDefects)
        self.threshold.editingFinished.connect(partial(self.identifyDefects, update=True))

        self.actionControls.triggered.connect(self.showOptions)
        self.actionSave.triggered.connect(self.saveImage)
        self.actionLoad.triggered.connect(self.loadImage)
        self.showOptions(False)

        commands = {self.speedBox.editingFinished : lambda: f"S:{self.speedBox.value()}",
                    self.speedSlider.sliderReleased : lambda: f"S:{self.speedBox.value()}",
                    self.xPosButton.clicked : lambda: f"X:{self.xBox.value()}",
                    self.xNegButton.clicked : lambda: f"X:-{self.xBox.value()}",
                    self.yPosButton.clicked : lambda: f"Y:{self.yBox.value()}",
                    self.yNegButton.clicked : lambda: f"Y:-{self.yBox.value()}",
                    self.zPosButton.clicked : lambda: f"Z:{self.zBox.value()}",
                    self.zNegButton.clicked : lambda: f"Z:-{self.zBox.value()}",
                    self.pos90Button.clicked : lambda: "R:90",
                    self.neg90Button.clicked : lambda: "R:-90",
                    self.pos45Button.clicked : lambda: "R:45",
                    self.neg45Button.clicked : lambda: "R:-45",
                    self.grabberAngleBox.editingFinished : lambda: f"A:{self.grabberAngleBox.value()}",
                    self.grabberAngleSlider.sliderReleased : lambda: f"A:{self.grabberAngleBox.value()}",
                    self.grabButton.clicked : lambda: "G:1",
                    self.releaseButton.clicked : lambda: "G:0"}

        for connection, command in commands.items():
            connection.connect(partial(self.serialWrite, command))
        
        self.blank = QPixmap("blank.png")
        self.segmentation_map, self.defect_image, self.image_path = None, None, None
        self.setDisplay(self.blank)

        self.serial = QSerialPort(self)
        self.serial.readyRead.connect(self.readSerialData)
        self.buffer, self.com_port, self.serial_port = b'', '', ''
        self.ports = []

        self.findPorts(close=True)
        self.menuSerial.aboutToShow.connect(self.findPorts)
        
    def findPorts(self, close=False):
        self.ports = sorted(serial.tools.list_ports.comports())
        actions = self.menuSerial.actions()
        existing_ports=[]
        for action in actions:
            existing_ports.append(action.text())
        
        for port in self.ports:
            if port.description in existing_ports:
                if close:
                    self.serial.setPortName(port[0])
                    self.serial.close()
                continue
            com_port = QAction(port.description, self, checkable=True)
            com_port.setObjectName(port.device)
            com_port.triggered.connect(self.onSerial)
            self.menuSerial.addAction(com_port)
            self.serial.setPortName(port[0])
            self.serial.close()        
            
    def setDisplay(self, pixmap: QPixmap) -> None:
        scaled = pixmap.scaled(self.display.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.display.setPixmap(scaled)
        self.display.setAlignment(Qt.AlignCenter)
    
    def onSerial(self):
        for port in self.menuSerial.actions():
            try:
                if port.isChecked():
                    self.com_port = port.objectName()
                    self.serial.setPortName(self.com_port) 
                    self.serial.setBaudRate(QSerialPort.Baud9600)
                    self.serial.setDataBits(QSerialPort.Data8)
                    self.serial.setParity(QSerialPort.NoParity)
                    self.serial.setStopBits(QSerialPort.OneStop)
                    self.serial.setFlowControl(QSerialPort.NoFlowControl)
                if not self.serial.open(QIODevice.ReadWrite) or not port.isChecked():
                    port.setChecked(False)
                    self.serial.close()
                    print(f"Closed port: {port.text()}".ljust(200))
                else:
                    print(f"Opened port: {port.text()}".ljust(200))
                    break
            except Exception as e:
                port.setChecked(False)

    def serialWrite(self, command_bind):
        command = command_bind()
        if self.serial.open(QIODevice.ReadWrite):
            self.serial.write(f'{command}\n'.encode('utf_8'))
        else:
            print(f"Serial port not opened! Command: {command} not sent.")
    
    @Slot()
    def readSerialData(self):
        try:
            incoming_data = self.serial.readAll().data()
            if not incoming_data:
                return
            self.buffer += incoming_data
            while b'\n' in self.buffer:
                line, self.buffer = self.buffer.split(b'\n', 1)
                decoded_data = line.decode('utf_8')
                print(decoded_data)
        except Exception as e:
            print(f"Error reading serial data: {e}")
            
    @Slot()
    def identifyDefects(self, update=False):
        threshold = self.threshold.value()
        model = self.models.currentText()
        model = "mvtec_"+model.replace(" ", "_")

        if self.image_path is not None and not update:
            print(f"Using {model} for image {os.path.basename(self.image_path)}")
            self.segmentation_map, self.defect_image = detectDefects(image_path=self.image_path, model_path=model)

        if None not in [self.segmentation_map, self.defect_image]:
            pixmap = drawDefects(self.segmentation_map, self.defect_image, threshold)
            self.setDisplay(pixmap)
        
    @Slot()
    def showOptions(self, checked: bool) -> None:
        if checked:    
            self.XYZFrame.show()
            self.pedestalFrame.show()
            self.grabberFrame.show()
        else:
            self.XYZFrame.hide()
            self.pedestalFrame.hide()
            self.grabberFrame.hide()
            
    @Slot()
    def setSpeed(self) -> None:
        self.speedSlider.setValue(self.speedBox.value())

    @Slot()
    def setGrabberAngle(self) -> None:
        self.grabberAngleSlider.setValue(self.grabberAngleBox.value())

    def loadImage(self) -> None:
        file_path, _ = QFileDialog.getOpenFileName(self, "Load Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if file_path:
            image = cv2.imread(file_path, cv2.IMREAD_COLOR)
            self.image_path = file_path
            self.defect_image = None
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channels = image.shape
            bytes_per_line = channels * width
            q_img = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_img)
            
            self.setDisplay(pixmap)
            print("Loading File:", file_path)

    def saveImage(self) -> None:
        file_path, _ = QFileDialog.getSaveFileName(None, "Save Image", "", "*.png;;*.jpg;;*.jpeg")
        if file_path:
            qimg = self.display.pixmap().toImage()
            qimg = qimg.convertToFormat(QImage.Format.Format_RGBA8888)
            
            width = qimg.width()
            height = qimg.height()
            arr = np.array(qimg.bits(), dtype=np.uint8).reshape(height, width, 4)
            image = cv2.cvtColor(arr, cv2.COLOR_RGBA2BGRA)

            cv2.imwrite(file_path, image)
            print("Saving File:", file_path)

    def closeEvent(self, event) -> None:
        try:
            event.accept() 
            super().closeEvent(event)
        except Exception as e:
            msg = f"Error during close event: {e}"
            print(msg)
            event.ignore()
        print('\nExited')

if not QApplication.instance():
    app = QApplication(sys.argv)
else:
    app = QApplication.instance()

if __name__ == '__main__':
    window = MainWindow()
    app.setStyle('Windows')
    window.show()
    print('Running\n')
    app.exec()

C:\Users\Nolan\anaconda3\envs\sandia_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running

Loading File: C:/Users/Nolan/Documents/Python Scripts/ME 266K/PyTorch/dataset/tile/test/oil/011.png
Using mvtec_tile for image 011.png


C:\Users\Nolan\anaconda3\envs\sandia_new\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Nolan\anaconda3\envs\sandia_new\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Exited


In [10]:
import cv2
import numpy as np

ip = "rtsp://192.168.1.19:8080/h264.sdp"  # Replace with the IP address from IP Webcam
cap = cv2.VideoCapture(ip)
cv2.namedWindow("Capturing", cv2.WINDOW_NORMAL)

if not cap.isOpened():
    print("Error: Could not open video stream from IP Webcam.")
    exit()

while True:
    ret, frame = cap.read()
    
    if frame is not None:
        cv2.resizeWindow("Capturing", 1280, 606)
        cv2.imshow("Capturing",frame)
    else:
        break
        print("Empty VideoCapture!")

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
cap.release()
cv2.destroyAllWindows()

In [13]:
import cv2
import numpy as np
import urllib.request

# URL to capture a photo from your IP Webcam
photo_url = "http://192.168.1.19:8080/photo.jpg"  # Adjust IP and port if needed

# Fetch the photo
try:
    with urllib.request.urlopen(photo_url) as response:
        image_data = response.read()
except Exception as e:
    print("Error fetching image:", e)
    exit()

# Convert the raw data to a NumPy array and decode into an OpenCV image
img_array = np.asarray(bytearray(image_data), dtype=np.uint8)
frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

if frame is not None:
    # Save the image at highest JPEG quality
    saved = cv2.imwrite("captured_image.jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    if saved:
        print("Image saved successfully at highest quality.")
    else:
        print("Failed to save image.")
else:
    print("Failed to decode image.")

Image saved successfully at highest quality.


In [1]:
pip freeze

anyio @ file:///home/conda/feedstock_root/build_artifacts/bld/rattler-build_anyio_1742243108/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1733311059102/work
argon2-cffi-bindings @ file:///D:/bld/argon2-cffi-bindings_1725356679565/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1733584251875/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1733250440834/work
async-lru @ file:///home/conda/feedstock_root/build_artifacts/bld/rattler-build_async-lru_1742153708/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1741918516150/work
babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1738490167835/work
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1738740337718/work
bleach @ file:///home/conda/feedstock_root/build_artifacts/bld/rattler-build_bleach_1737382993/work
Brotli @ file:///D:/bld/brotli-split_1725267609074/work
cached-property @ file:///